<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/colab_FH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
We first import the data as a pandas dataframe. Then we create a list made up of the title and genre columns from the original data. We also create a list including all 32 genres.

In order to import the data into Google Colab, we have to first upload the csv file:

In [1]:
from google.colab import files
uploaded = files.upload()

Saving book32listing.csv to book32listing.csv


In [2]:
import pandas as pd
df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [3]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [4]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [5]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [6]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating a translation table. 
Punctuation to be removed is given in string: string.punctuation

In [7]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
This splits all titles into words (output: list of lists of strings)

In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams (could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.)

In [10]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [11]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

We first create an empty list to contain lists of stems in each title. The second empty list will contain the stem of the title. Each stemmed word in the title will be added to the second list. Each time the second list is filled, it will appended as a list to the first list. 

In [12]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming. 
We should test this out to see whether it makes a difference in final classifier

In [13]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [14]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title. This uses postags created in previous step

In [15]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?


-Naive Bayes (bag-of-words)

-BERT (language model)

https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794

## Split data into train/test

In order to split the data into train and test sets, we create a dataframe containing the tokenized titles. We then split the data into 80% for training and 20% for testing. 

In [16]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [17]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

## Naive Bayes classifier

We build the Naive Bayes classifier from scratch. In order to do this, we create lists of the following: token frequencies in total vocabulary, token frequencies in each genre, number of words in each genre, number of words in total vocabulary, number of total titles, number of titles in each genre. Using these, we compute the priors for each genre and the likelihoods for each word to be in any genre. 
Using both the priors and the likelihoods, we create a dataframe with prediction values. By taking the highest prediction value of a particular word across the genres, we can get the predicted genre for that word.

We then test out whether the model works for the test data.

### Train model: Prep

In [18]:
from collections import Counter

In [19]:
# token frequencies within each title (we don't need this...)
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [20]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92423, 'and': 54286, 'of': 51099, 'a': 31379, 'to': 27838, 'for': 21394, 'in': 21211, 'guide': 16013, 'edition': 12503, 'book': 8084, 'with': 7837, 'your': 7359, 'from': 6543, 'series': 6283, 'on': 5929, 'how': 5831, 'an': 5120, 'new': 4783, 'life': 4679, 'world': 4044, 'volume': 4017, 'history': 3998, 'you': 3836, '1': 3313, 'recipes': 3210, 'by': 3131, 'american': 3028, 'travel': 2932, 'complete': 2776, 'law': 2717, '2': 2692, 'story': 2662, 'art': 2584, 'handbook': 2511, 'health': 2484, 'my': 2459, 'books': 2396, 'science': 2242, 'at': 2200, 'calendar': 2196, 'novel': 2081, 'best': 2060, 'practice': 1999, 'i': 1968, 'cookbook': 1938, 'map': 1909, 'guides': 1894, 'that': 1891, 'about': 1826, 'love': 1819, 'vol': 1813, 'what': 1748, '2016': 1727, 'great': 1696, 'stories': 1675, 'business': 1629, 'more': 1624, 'war': 1622, 'is': 1584, 'it': 1578, 'one': 1558, 'manual': 1558, 'introduction': 1557, 'modern': 1552, 'america': 1549, 'second': 1517, 'management': 1509, 'food

In [21]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [22]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_title = dict(genre_title)
    genre_count.append(genre_title)

In [23]:
# token frequencies in each genre
genre_count[0] # genres in same order as GenreGroups

{'moms': 10,
 'family': 10,
 'wall': 784,
 'calendar': 1955,
 '2016': 1095,
 'doug': 1,
 'the': 387,
 'pug': 3,
 'moleskine': 29,
 'weekly': 66,
 'notebook': 18,
 '12m': 6,
 'large': 25,
 'black': 27,
 'soft': 7,
 'cover': 28,
 '5': 14,
 'x': 27,
 '825': 13,
 '365': 80,
 'cats': 41,
 'color': 10,
 'pageaday': 56,
 'sierra': 3,
 'club': 5,
 'engagement': 66,
 'wilderness': 9,
 'ansel': 3,
 'adams': 3,
 'dilbert': 5,
 'daytoday': 89,
 'cat': 26,
 'gallery': 13,
 'llewellyns': 6,
 'witches': 3,
 'datebook': 3,
 'amy': 3,
 'knapp': 2,
 'big': 11,
 'grid': 3,
 'essential': 1,
 'organization': 1,
 'and': 142,
 'communication': 1,
 'tool': 1,
 'for': 70,
 'entire': 1,
 'audubon': 7,
 'nature': 13,
 'national': 15,
 'park': 3,
 'foundation': 1,
 'mary': 6,
 'engelbreit': 5,
 'enjoy': 1,
 'joy': 1,
 'grumpy': 1,
 'extra': 3,
 '75': 3,
 '10': 3,
 'susan': 4,
 'branch': 4,
 'dogs': 6,
 'maxine': 5,
 'yearinabox': 5,
 'disney': 21,
 'descendants': 1,
 'astrological': 1,
 '83rd': 1,
 'edition': 121

In [24]:
# number of words in a class
len(genre_count[0]) # first genre

2393

In [25]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74713


In [26]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

166001


In [27]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14658
Children's Books             10797
Medical Books                 9649
Health, Fitness & Dieting     9544
Business & Money              7976
Name: genres, dtype: int64


### Priors

In [28]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08830067288751273), ('Comics & Graphic Novels', 0.06504177685676592), ('Test Preparation', 0.05812615586653092), ('Mystery, Thriller & Suspense', 0.057493629556448454), ('Science Fiction & Fantasy', 0.04804790332588358)]


### Likelihoods

In [29]:
# likelihoods of each word (probability of word being in specific genre)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

len_gc = -1
for i in genre_count: # loop through genres
    len_gc += 1 # create genre index
    for word in i: 
        p = (genre_count[len_gc][word] + 1) / (len(genre_count[len_gc]) + V) # probability
        likelihood.loc[word, likelihood.columns[len_gc]] = p # replace NaN in dataframe with p

In [30]:
# now fill likelihoods of words that haven't appeared in genre

len_gc = -1
for c, v in likelihood.iteritems(): # loop through genres in dataframe
  len_gc += 1 # create genre index
  p = 1 / (len(genre_count[len_gc]) + V) # probability of word not appearing in that genre
  likelihood[c].fillna(p, inplace=True) # replace remaining NaNs in dataframe with p

In [31]:
likelihood

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,0.000143,0.000026,0.000013,0.000013,0.000013,0.000013,0.000060,0.000012,0.000052,0.000013,0.000156,0.000047,0.000036,0.000039,0.000343,0.000038,0.000012,0.000024,0.000024,0.000116,0.000012,0.000024,0.000024,0.000012,0.000025,0.000025,0.000096,0.000036,0.000024,0.000012,0.000047,0.000034
family,0.000143,0.000153,0.000181,0.000103,0.000038,0.000429,0.000394,0.000409,0.000387,0.000050,0.002287,0.000469,0.000277,0.000142,0.001586,0.000241,0.000098,0.000593,0.000202,0.001346,0.000773,0.001170,0.000767,0.000261,0.000188,0.000389,0.000963,0.000347,0.000109,0.000337,0.000643,0.001068
wall,0.010181,0.000013,0.000026,0.000077,0.000063,0.000013,0.000215,0.000048,0.000013,0.000038,0.000036,0.000246,0.000289,0.000026,0.000013,0.000025,0.000037,0.000059,0.000095,0.000058,0.000761,0.000073,0.000122,0.000154,0.000025,0.000176,0.000108,0.000155,0.000158,0.000084,0.000211,0.000443
calendar,0.025368,0.000013,0.000013,0.000013,0.000013,0.000025,0.000311,0.000048,0.000013,0.000164,0.000048,0.000340,0.000495,0.000013,0.000063,0.000076,0.000074,0.000012,0.000142,0.000058,0.000059,0.000012,0.000012,0.000012,0.000050,0.000050,0.000120,0.000275,0.000243,0.000024,0.000058,0.000409
2016,0.014214,0.000013,0.001226,0.000013,0.000025,0.000013,0.000371,0.000060,0.000039,0.000453,0.000060,0.000446,0.000507,0.000193,0.000102,0.000127,0.000552,0.000557,0.000107,0.000081,0.000297,0.000110,0.000024,0.000036,0.000063,0.000364,0.000144,0.000287,0.000255,0.000012,0.000105,0.001386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
flashmaps,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
ballrooms,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
riverboat,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
historymapped,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023


In [32]:
# predictions on word by word basis using words in titles of test data

predictors = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq)) # create empty dataframe
len_gc = -1
for i in prior:
  len_gc += 1 # create genre index
  # square all likelihoods then multiply them each with prior of that genre
  predictors = likelihood.loc[:, df1.genre.unique().tolist()] ** 2 # square likelihoods
  predictors = predictors.loc[:, df1.genre.unique().tolist()] * np.array(prior[len_gc][1]) # multiply by priors

In [33]:
predictors

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.330234e-10,4.251227e-12,1.088554e-12,1.083264e-12,1.040058e-12,1.039271e-12,2.334435e-11,9.469478e-13,1.737379e-11,1.034049e-12,1.600612e-10,1.439235e-11,8.558342e-12,9.767471e-12,7.674095e-10,9.453066e-12,9.820185e-13,3.672151e-12,3.678339e-12,8.794554e-11,9.237217e-13,3.885949e-12,3.872435e-12,9.200212e-13,4.086496e-12,4.125707e-12,6.057258e-11,8.403163e-12,3.863027e-12,9.495519e-13,1.431500e-11,7.594298e-12
family,1.330234e-10,1.530442e-10,2.133567e-10,6.932887e-11,9.360520e-12,1.201397e-09,1.016880e-09,1.094672e-09,9.772759e-10,1.654478e-11,3.419058e-08,1.439235e-09,5.030403e-10,1.313182e-10,1.644825e-08,3.791730e-10,6.284919e-11,2.295095e-09,2.657600e-10,1.183395e-08,3.902724e-09,8.953227e-09,3.842424e-09,4.452903e-10,2.298654e-10,9.912010e-10,6.057258e-09,7.852289e-10,7.822629e-11,7.444487e-10,2.706429e-09,7.455913e-09
wall,6.774574e-07,1.062807e-12,4.354218e-12,3.899749e-11,2.600145e-11,1.039271e-12,3.025428e-10,1.515117e-11,1.085862e-12,9.306440e-12,8.523967e-12,3.966892e-10,5.477339e-10,4.341098e-12,1.052688e-12,4.201363e-12,8.838167e-12,2.295095e-11,5.885342e-11,2.198638e-11,3.783564e-09,3.497354e-11,9.681088e-11,1.554836e-10,4.086496e-12,2.021596e-10,7.666217e-11,1.577927e-10,1.632129e-10,4.652805e-11,2.898787e-10,1.283436e-09
calendar,4.206108e-06,1.062807e-12,1.088554e-12,1.083264e-12,1.040058e-12,4.157084e-12,6.312312e-10,1.515117e-11,1.085862e-12,1.747543e-10,1.515372e-11,7.564980e-10,1.598508e-09,1.085275e-12,2.631720e-11,3.781226e-11,3.535267e-11,9.180378e-13,1.324202e-10,2.198638e-11,2.309304e-11,9.714873e-13,9.681088e-13,9.200212e-13,1.634598e-11,1.650283e-11,9.464465e-11,4.939192e-10,3.863027e-10,3.798208e-12,2.236718e-11,1.093579e-09
2016,1.320577e-06,1.062807e-12,9.824203e-09,1.083264e-12,4.160231e-12,1.039271e-12,8.973568e-10,2.367370e-11,9.772759e-12,1.340127e-09,2.367769e-11,1.298910e-09,1.677435e-09,2.441868e-10,6.737203e-11,1.050341e-10,1.988588e-09,2.027946e-09,7.448637e-11,4.309331e-11,5.773261e-10,7.869047e-11,3.872435e-12,8.280191e-12,2.554060e-11,8.674298e-10,1.362883e-10,5.378024e-10,4.258987e-10,9.495519e-13,7.246968e-11,1.255928e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
flashmaps,1.099367e-12,1.062807e-12,1.088554e-12,1.083264e-12,1.040058e-12,1.039271e-12,9.337740e-13,9.469478e-13,1.085862e-12,1.034049e-12,9.471074e-13,8.995220e-13,9.509268e-13,1.085275e-12,1.052688e-12,1.050341e-12,9.820185e-13,9.180378e-13,9.195848e-13,8.794554e-13,9.237217e-13,9.714873e-13,9.681088e-13,9.200212e-13,1.021624e-12,1.031427e-12,9.464465e-13,9.336847e-13,9.657567e-13,9.495519e-13,8.946874e-13,3.375244e-12
ballrooms,1.099367e-12,1.062807e-12,1.088554e-12,1.083264e-12,1.040058e-12,1.039271e-12,9.337740e-13,9.469478e-13,1.085862e-12,1.034049e-12,9.471074e-13,8.995220e-13,9.509268e-13,1.085275e-12,1.052688e-12,1.050341e-12,9.820185e-13,9.180378e-13,9.195848e-13,8.794554e-13,9.237217e-13,9.714873e-13,9.681088e-13,9.200212e-13,1.021624e-12,1.031427e-12,9.464465e-13,9.336847e-13,9.657567e-13,9.495519e-13,8.946874e-13,3.375244e-12
riverboat,1.099367e-12,1.062807e-12,1.088554e-12,1.083264e-12,1.040058e-12,1.039271e-12,9.337740e-13,9.469478e-13,1.085862e-12,1.034049e-12,9.471074e-13,8.995220e-13,9.509268e-13,1.085275e-12,1.052688e-12,1.050341e-12,9.820185e-13,9.180378e-13,9.195848e-13,8.794554e-13,9.237217e-13,9.714873e-13,9.

In [34]:
predictors.loc[["moms"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.330234e-10,4.251227e-12,1.088554e-12,1.083264e-12,1.040058e-12,1.039271e-12,2.334435e-11,9.469478e-13,1.737379e-11,1.034049e-12,1.600612e-10,1.439235e-11,8.558342e-12,9.767471e-12,7.674095e-10,9.453066e-12,9.820185e-13,3.672151e-12,3.678339e-12,8.794554e-11,9.237217e-13,3.885949e-12,3.872435e-12,9.200212e-13,4.086496e-12,4.125707e-12,6.057258e-11,8.403163e-12,3.863027e-12,9.495519e-13,1.431500e-11,7.594298e-12


In [180]:
# output genre that has highest prediction value for input word
maxValueIndexObj = predictors.idxmax(axis=1)
print(list(maxValueIndexObj[["high"]]))
print(list(maxValueIndexObj[["higher"]]))
print(list(maxValueIndexObj[["highest"]]))

['Health, Fitness & Dieting']
['Education & Teaching']
['Christian Books & Bibles']


In [181]:
# function that will return list of predicted genres based on title input
def pred_genre(title):
  prediction = []
  for w in title:
    if w in predictors.index:
      maxValueIndexObj = predictors.idxmax(axis=1)
      prediction += list(maxValueIndexObj[[w]])
    if w not in predictors.index:
      pass
    # for each genre have values of 0-1 how high prediction is

  return prediction 
  #return list(set(prediction)) # remove duplicate genres

In [157]:
# predicted genres for first test title
pred_genre(test['titles'][0])

["Children's Books",
 'Calendars',
 'Travel',
 "Children's Books",
 'Self-Help',
 'Cookbooks, Food & Wine',
 'Calendars',
 'Calendars',
 'Calendars']

In [190]:
dc = dict(Counter(pred_genre(test['titles'][0])))

In [176]:
len(pred_genre(test['titles'][0]))

9

In [193]:
for c in dc.values():
  print(c)
  probab = c / len(pred_genre(test['titles'][0]))
  dc.update([probab]) # update count with probability of prediction
print(probab)

2


TypeError: ignored

In [96]:
# "real" genre of first test title
test['genres'][0]

'Calendars'

In [140]:
len(test)

41571

In [155]:
# loop across all test titles to get prediction accuracy
preds = []
for t in range(len(test)):
  pred = pred_genre(test['titles'][t])
  preds.append(pred_genre(test['titles'][t]))

KeyboardInterrupt: ignored

In [ ]:
# loop across all test title:
# create list of predicted genres
# check whether real genre is included in prediction
# this would give accuracy?

### Test model

We test out, whether the model works for the test dataset. This should give an accuracy value in order to evaluate the model and later compare it to the BERT classifier.

In [ ]:
# use test data to output genre
# look at each word separately -> give prediction value of that genre
# output multiple genres?